In [1]:
import pandas as pd

In [2]:
!pip install openpyxl

In [3]:
import pandas as pd

# Open the Excel file
df = pd.read_excel('cordinates.xlsx')



In [4]:
df

,location,X,Y
0,آتش‌نشانی,696063.6492,4.050009e+06
1,آتش‌نشانی و کلانتری,693700.1689,4.049392e+06
2,اسکان موقت_1,690301.8602,4.047174e+06
3,اسکان موقت_2,692509.7822,4.048808e+06
4,انبار و سردخانه,693888.4630,4.052246e+06
5,باغ گیاه شناسی,689035.1816,4.047994e+06
6,بخشداری،پاسگاه و پایگاه بسیج,697876.7896,4.050822e+06
7,بهداشتی-درمانی(پیشنهادی),694103.4791,4.047709e+06
8,بیمارستان,697548.9062,4.050948e+06
9,بیمارستان موجود,695828.3050,4.050435e+06


In [12]:
!pip install pyproj

  Using cached pyproj-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.3 MB ? eta -:--:--0m
error: incomplete-download

× Download failed because not enough bytes were received (0 bytes/9.3 MB)
╰─> URL: https://files.pythonhosted.org/packages/76/a5/c6e11b9a99ce146741fb4d184d5c468446c6d6015b183cae82ac822a6cfa/pyproj-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

note: This is an issue with network connectivity, not pip.
hint: Consider using --resume-retries to enable download resumption.


In [5]:
import pandas as pd
import folium
from pyproj import Transformer

# Read the Excel file
df = pd.read_excel('cordinates.xlsx')

# Create a Transformer for UTM Zone 40 North to WGS84 (lat/lon)
transformer = Transformer.from_crs("EPSG:32640", "EPSG:4326", always_xy=True)

# Function to convert UTM to decimal degrees
def utm_to_latlon(row):
    if pd.isna(row['X']) or pd.isna(row['Y']):  # Handle NaN values
        return pd.Series([None, None], index=['latitude', 'longitude'])
    lon, lat = transformer.transform(row['X'], row['Y'])
    return pd.Series([lat, lon], index=['latitude', 'longitude'])

# Apply conversion to each row
df[['latitude', 'longitude']] = df.apply(utm_to_latlon, axis=1)

# Create a Folium map centered on the average of valid coordinates
valid_coords = df.dropna(subset=['latitude', 'longitude'])
center_lat = valid_coords['latitude'].mean()
center_lon = valid_coords['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for each valid location
for _, row in valid_coords.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['location'],
        tooltip=row['location']
    ).add_to(m)

# Save the map to an HTML file
m.save('coordinates_map.html')

# Print the DataFrame to verify coordinates
print(df)

# Inform user about the map
print("Map has been saved as 'coordinates_map.html'. Open it in a web browser to view the locations.")

                        location            X             Y   latitude  \
0                     آتش‌نشانی   696063.6492  4.050009e+06  36.575420   
1            آتش‌نشانی و کلانتری  693700.1689  4.049392e+06  36.570350   
2                   اسکان موقت_1  690301.8602  4.047174e+06  36.551045   
3                   اسکان موقت_2  692509.7822  4.048808e+06  36.565324   
4                انبار و سردخانه  693888.4630  4.052246e+06  36.596019   
5                 باغ گیاه شناسی  689035.1816  4.047994e+06  36.558685   
6   بخشداری،پاسگاه و پایگاه بسیج  697876.7896  4.050822e+06  36.582369   
7       بهداشتی-درمانی(پیشنهادی)  694103.4791  4.047709e+06  36.555100   
8                      بیمارستان  697548.9062  4.050948e+06  36.583577   
9                بیمارستان موجود  695828.3050  4.050435e+06  36.579305   
10           بیمارستان(پیشنهادی)  689236.0929  4.047212e+06  36.551604   
11                        پایانه  691577.7646  4.047074e+06  36.549888   
12                    پایانه بار  6940

In [6]:
m

In [5]:
df.to_excel('converted_cordinates.xlsx', index = False)


In [7]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('converted_cordinates.xlsx')

# Function to format location name (replace spaces and special characters with underscores)
def format_location_name(location):
    return location.replace(' ', '_').replace('(', '_').replace(')', '')

# Filter out rows with NaN values in latitude or longitude
valid_df = df #.dropna(subset=['latitude', 'longitude'])

# Create the points string in the desired format
points = "points=(\n"
for _, row in valid_df.iterrows():
    location = format_location_name(row['location'])
    points += f'    "{location} {row["longitude"]} {row["latitude"]}"\n'
points += ")"

# Print the result
print(points)

# Save the points string to a text file
with open('points_template.txt', 'w', encoding='utf-8') as f:
    f.write(points)

points=(
    "آتش‌نشانی_ 59.1912743731235 36.5754201728548"
    "آتش‌نشانی_و_کلانتری 59.164721438337 36.5703504724028"
    "اسکان_موقت_1 59.1262190392515 36.5510450549245"
    "اسکان_موقت_2 59.1512804779774 36.565323611865"
    "انبار_و_سردخانه 59.1675432946918 36.5960192021979"
    "باغ_گیاه_شناسی 59.1122764837332 36.5586846037151"
    "بخشداری،پاسگاه_و_پایگاه_بسیج 59.2117340048099 36.5823688346489"
    "بهداشتی-درمانی_پیشنهادی 59.1688015123926 36.5550996150264"
    "بیمارستان 59.2081040976203 36.5835769255997"
    "بیمارستان_موجود 59.1887541634192 36.5793051817252"
    "بیمارستان_پیشنهادی 59.1143281818572 36.5516042714619"
    "پایانه 59.1404407063252 36.5498881289438"
    "پایانه_بار 59.1692644291791 36.6001698489115"
    "پایانه_مسافربری_چند_منظوره 59.2052974209545 36.5857663906958"
    "پست_برق 59.1728054806605 36.575111978516"
    "پلیس_راهور 59.1787168955991 36.6071806616717"
    "جایگاه_سوخت 59.2152136730854 36.5803734054848"
    "دانشگاه 59.155860912244 36.5555437148639"
    "

In [8]:
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import matplotlib.dates as mdates

# Set global font to Times New Roman
#plt.rcParams['font.family'] = 'Times New Roman'

# Directory containing time series files
directory = "./time_series"

# Function to parse date string to datetime
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y%m%d')
    except ValueError as e:
        print(f"Error parsing date '{date_str}': {e}")
        raise

# Function to process a single file and create a plot
def process_file(file_path, filename):
    print(f"Processing file: {file_path}")
    try:
        # Read the file
        dates = []
        displacements = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            print(f"Read {len(lines)} lines from {filename}")
            # Start reading data after the last '#' line
            data_start = False
            for line in lines:
                if line.startswith('#'):
                    data_start = True
                    continue
                if data_start and line.strip():
                    try:
                        date_str, displacement = line.strip().split()
                        dates.append(parse_date(date_str))
                        displacements.append(float(displacement))
                    except ValueError as e:
                        print(f"Error parsing line '{line.strip()}' in {filename}: {e}")
                        continue

        if not dates:
            print(f"No valid data found in {filename}")
            return

        print(f"Parsed {len(dates)} data points from {filename}")

        # Convert dates to matplotlib date numbers
        dates_mpl = mdates.date2num(dates)
        X = np.array(dates_mpl).reshape(-1, 1)
        y = np.array(displacements)

        # Fit linear model
        model = LinearRegression().fit(X, y)
        slope = model.coef_[0]
        intercept = model.intercept_
        trendline = model.predict(X)
        annual_rate = int(round(slope * 365))  # mm/year, without decimals
        print(f"Linear model for {filename}: y = {slope:.2f}x + {intercept:.2f}, Annual Rate: {annual_rate} mm/year")

        # Create the plot
        plt.figure(figsize=(14, 4))
        # Scatter plot with black outline
        plt.scatter(dates, y, color='red', label='InSAR Data', alpha=0.8, s=80, edgecolors='black', linewidth=0.3)
        plt.plot(dates, trendline, color='black', linestyle='--', alpha=0.6, linewidth=2.5, label=f'Trendline: y = {slope:.2f}x + {intercept:.2f}\nAnnual Rate: {annual_rate} mm/year')

        # Set y-axis limits
        plt.ylim(-130, 30)

        # Labels and formatting
        plt.ylabel("Displacement (mm)", fontsize=14)
        plt.title(f"Subsidence", fontsize=15)
        plt.grid(True)
        # Place legend with trendline info
        plt.legend(fontsize=16, loc='best')

        # Rotate and bold x-axis tick labels
        plt.xticks(rotation=45)
        # for label in plt.gca().get_xticklabels():
        #     label.set_fontweight('bold')

        # Format x-axis to show only years
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())

        # Adjust layout to prevent label cutoff
        plt.tight_layout()

        # Save the plot
        output_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}.png")
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Saved plot to: {output_path}")
        plt.close()

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Check if directory exists
if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist")
else:
    print(f"Scanning directory: {directory}")
    # Iterate through all files in the time_series directory
    file_count = 0
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_count += 1
            file_path = os.path.join(directory, filename)
            print(f"Found file: {filename}")
            process_file(file_path, filename)
    if file_count == 0:
        print(f"No .txt files found in {directory}")

Scanning directory: ./time_series
Found file: شورای_حل_اختلاف.txt
Processing file: ./time_series/شورای_حل_اختلاف.txt
Read 239 lines from شورای_حل_اختلاف.txt
Parsed 231 data points from شورای_حل_اختلاف.txt
Linear model for شورای_حل_اختلاف.txt: y = 0.00x + -59.92, Annual Rate: 1 mm/year
Saved plot to: ./time_series/شورای_حل_اختلاف.png
Found file: مجتمع_آموزشی_پرواز.txt
Processing file: ./time_series/مجتمع_آموزشی_پرواز.txt
Read 239 lines from مجتمع_آموزشی_پرواز.txt
Parsed 231 data points from مجتمع_آموزشی_پرواز.txt
Linear model for مجتمع_آموزشی_پرواز.txt: y = -0.01x + 247.66, Annual Rate: -5 mm/year
Saved plot to: ./time_series/مجتمع_آموزشی_پرواز.png
Found file: مجتمع_آموزشی_پرواز_موجود.txt
Processing file: ./time_series/مجتمع_آموزشی_پرواز_موجود.txt
Read 239 lines from مجتمع_آموزشی_پرواز_موجود.txt
Parsed 231 data points from مجتمع_آموزشی_پرواز_موجود.txt
Linear model for مجتمع_آموزشی_پرواز_موجود.txt: y = -0.00x + 78.55, Annual Rate: -2 mm/year
Saved plot to: ./time_series/مجتمع_آموزشی_پروا

In [11]:
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import matplotlib.dates as mdates

# Set global font to Times New Roman
#plt.rcParams['font.family'] = 'Times New Roman'

# Directory containing time series files
directory = "./time_series"

# Function to parse date string to datetime
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y%m%d')
    except ValueError as e:
        print(f"Error parsing date '{date_str}': {e}")
        raise

# Function to process a single file and create a plot
def process_file(file_path, filename):
    print(f"Processing file: {file_path}")
    try:
        # Read the file
        dates = []
        displacements = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            print(f"Read {len(lines)} lines from {filename}")
            # Start reading data after the last '#' line
            data_start = False
            for line in lines:
                if line.startswith('#'):
                    data_start = True
                    continue
                if data_start and line.strip():
                    try:
                        date_str, displacement = line.strip().split()
                        dates.append(parse_date(date_str))
                        displacements.append(float(displacement))
                    except ValueError as e:
                        print(f"Error parsing line '{line.strip()}' in {filename}: {e}")
                        continue

        if not dates:
            print(f"No valid data found in {filename}")
            return

        print(f"Parsed {len(dates)} data points from {filename}")

        # Convert dates to matplotlib date numbers
        dates_mpl = mdates.date2num(dates)
        X = np.array(dates_mpl).reshape(-1, 1)
        y = np.array(displacements)

        # Fit linear model
        model = LinearRegression().fit(X, y)
        slope = model.coef_[0]
        intercept = model.intercept_
        trendline = model.predict(X)
        annual_rate = int(round(slope * 365))  # mm/year, without decimals
        print(f"Linear model for {filename}: y = {slope:.2f}x + {intercept:.2f}, Annual Rate: {annual_rate} mm/year")

        # Create the plot
        plt.figure(figsize=(14, 4))
        # Scatter plot with black outline
        plt.scatter(dates, y, color='red', label='InSAR Data', alpha=0.8, s=80, edgecolors='black', linewidth=0.3)
     #   plt.plot(dates, trendline, color='black', linestyle='--', alpha=0.6, linewidth=2.5, label=f'Trendline: y = {slope:.2f}x + {intercept:.2f}\nAnnual Rate: {annual_rate} mm/year')
        
        plt.plot(dates, trendline, color='black', linestyle='--', alpha=0.6, linewidth=2.5, label=f'Annual Rate: {annual_rate} mm/year')
        # Add bold y=0 baseline
        plt.axhline(y=0, color='black', linewidth=1, alpha=0.8 ) #label='Baseline (y=0)')

        # Set y-axis limits
        plt.ylim(-130, 30)

        # Labels and formatting
        plt.ylabel("Displacement (mm)", fontsize=14)
        plt.title(f"Subsidence", fontsize=15)
        plt.grid(True)
        # Place legend with trendline info
        plt.legend(fontsize=16, loc='best')

        # Rotate and bold x-axis tick labels
        plt.xticks(rotation=45)
        # for label in plt.gca().get_xticklabels():
        #     label.set_fontweight('bold')

        # Format x-axis to show only years
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())

        # Adjust layout to prevent label cutoff
        plt.tight_layout()

        # Save the plot
        output_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}.png")
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Saved plot to: {output_path}")
        plt.close()

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Check if directory exists
if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist")
else:
    print(f"Scanning directory: {directory}")
    # Iterate through all files in the time_series directory
    file_count = 0
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_count += 1
            file_path = os.path.join(directory, filename)
            print(f"Found file: {filename}")
            process_file(file_path, filename)
    if file_count == 0:
        print(f"No .txt files found in {directory}")

Scanning directory: ./time_series
Found file: شورای_حل_اختلاف.txt
Processing file: ./time_series/شورای_حل_اختلاف.txt
Read 239 lines from شورای_حل_اختلاف.txt
Parsed 231 data points from شورای_حل_اختلاف.txt
Linear model for شورای_حل_اختلاف.txt: y = 0.00x + -59.92, Annual Rate: 1 mm/year
Saved plot to: ./time_series/شورای_حل_اختلاف.png
Found file: مجتمع_آموزشی_پرواز.txt
Processing file: ./time_series/مجتمع_آموزشی_پرواز.txt
Read 239 lines from مجتمع_آموزشی_پرواز.txt
Parsed 231 data points from مجتمع_آموزشی_پرواز.txt
Linear model for مجتمع_آموزشی_پرواز.txt: y = -0.01x + 247.66, Annual Rate: -5 mm/year
Saved plot to: ./time_series/مجتمع_آموزشی_پرواز.png
Found file: مجتمع_آموزشی_پرواز_موجود.txt
Processing file: ./time_series/مجتمع_آموزشی_پرواز_موجود.txt
Read 239 lines from مجتمع_آموزشی_پرواز_موجود.txt
Parsed 231 data points from مجتمع_آموزشی_پرواز_موجود.txt
Linear model for مجتمع_آموزشی_پرواز_موجود.txt: y = -0.00x + 78.55, Annual Rate: -2 mm/year
Saved plot to: ./time_series/مجتمع_آموزشی_پروا

In [ ]:
import pandas as pd
import folium
from pyproj import Transformer
import folium.plugins

# Read the Excel file
df = pd.read_excel('cordinates.xlsx')

# Create a Transformer for UTM Zone 40 North to WGS84 (lat/lon)
transformer = Transformer.from_crs("EPSG:32640", "EPSG:4326", always_xy=True)

# Function to convert UTM to decimal degrees
def utm_to_latlon(row):
    if pd.isna(row['X']) or pd.isna(row['Y']):  # Handle NaN values
        return pd.Series([None, None], index=['latitude', 'longitude'])
    lon, lat = transformer.transform(row['X'], row['Y'])
    return pd.Series([lat, lon], index=['latitude', 'longitude'])

# Apply conversion to each row
df[['latitude', 'longitude']] = df.apply(utm_to_latlon, axis=1)

# Create a Folium map centered on the average of valid coordinates
valid_coords = df.dropna(subset=['latitude', 'longitude'])
center_lat = valid_coords['latitude'].mean()
center_lon = valid_coords['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for each valid location
for _, row in valid_coords.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['location'],
        tooltip=row['location']
    ).add_to(m)

# Add measurement tool
folium.plugins.MeasureControl(
    position='topright',
    primary_length_unit='meters',
    secondary_length_unit='kilometers',
    primary_area_unit='sqmeters',
    secondary_area_unit='hectares'
).add_to(m)

# Save the map to an HTML file
m.save('coordinates_map.html')

# Print the DataFrame to verify coordinates
print(df)

# Inform user about the map
print("Map with measurement tool has been saved as 'coordinates_map.html'. Open it in a web browser to view the locations.")

In [9]:
def process_file(file_path, filename):
    print(f"\n=== Processing file: {filename} ===")
    try:
        # Read the file
        with open(file_path, 'r') as f:
            lines = f.readlines()

        print(f"Total lines read: {len(lines)}")

        # Find the line containing '# linear model:'
        slope = intercept = None
        for line in lines:
            if line.lower().startswith('# linear model:'):
                linear_model_line = line.strip()
                linear_eq = linear_model_line.split(":", 1)[1].strip()
                print(f"Found linear model: {linear_eq}")

                # Parse the linear equation
                import re
                match = re.match(r"([+-]?\d*\.?\d+)\*t\s*([+-]\s*\d*\.?\d+)", linear_eq.replace(" ", ""))
                if match:
                    slope = float(match.group(1))
                    intercept = float(match.group(2).replace(" ", ""))
                    print(f"Parsed slope: {slope}, intercept: {intercept}")
                else:
                    print(f"Could not parse linear model line: {linear_model_line}")
                break  # Only parse the first match

        if slope is None or intercept is None:
            print(f"Linear model not found or not parsed in {filename}. Skipping.")
            return

        # Read time series data (skip lines starting with '#')
        dates = []
        displacements = []
        for line in lines:
            if line.startswith('#') or not line.strip():
                continue
            try:
                date_str, displacement = line.strip().split()
                dates.append(parse_date(date_str))
                displacements.append(float(displacement))
            except ValueError as e:
                print(f"Skipping bad line: {line.strip()} ({e})")
                continue

        if not dates:
            print(f"No valid data found in {filename}")
            return

        print(f"Parsed {len(dates)} data points.")

        # Prepare for plotting
        dates_mpl = mdates.date2num(dates)
        X = np.array(dates_mpl).reshape(-1, 1)
        y = np.array(displacements)

        # Simulate trendline using extracted slope/intercept
        trendline = slope * (dates_mpl - dates_mpl[0]) + intercept
        annual_rate = int(round(slope * 365))

        print(f"Using trendline: y = {slope:.2f}*t + {intercept:.2f} | Annual Rate: {annual_rate} mm/year")

        # Plotting
        plt.figure(figsize=(14, 4))
        plt.scatter(dates, y, color='red', label='InSAR Data', alpha=0.8,
                    s=80, edgecolors='black', linewidth=0.3)
        plt.plot(dates, trendline, color='black', linestyle='--', alpha=0.6, linewidth=2.5,
                 label=f'Trendline: y = {slope:.2f}*t + {intercept:.2f}\nAnnual Rate: {annual_rate} mm/year')

        plt.ylim(-130, 30)
        plt.ylabel("Displacement (mm)", fontsize=14)
        plt.title("Subsidence", fontsize=15)
        plt.grid(True)
        plt.legend(fontsize=16, loc='best')
        plt.xticks(rotation=45)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        plt.tight_layout()

        output_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}.png")
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"✅ Saved plot to: {output_path}")
        plt.close()

    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")


In [10]:
directory = "./time_series"

if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist")
else:
    print(f"Scanning directory: {directory}")
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            print(f"Found file: {filename}")
            file_path = os.path.join(directory, filename)
            process_file(file_path, filename)


Scanning directory: ./time_series
Found file: آتش‌نشانی_.txt

=== Processing file: آتش‌نشانی_.txt ===
Total lines read: 239
Found linear model: 1.961*t+0.529
Parsed slope: 1.961, intercept: 0.529
Parsed 231 data points.
Using trendline: y = 1.96*t + 0.53 | Annual Rate: 716 mm/year
✅ Saved plot to: ./time_series\آتش‌نشانی_.png
Found file: آتش‌نشانی_و_کلانتری.txt

=== Processing file: آتش‌نشانی_و_کلانتری.txt ===
Total lines read: 239
Found linear model: 2.025*t-0.648
Parsed slope: 2.025, intercept: -0.648
Parsed 231 data points.
Using trendline: y = 2.02*t + -0.65 | Annual Rate: 739 mm/year
✅ Saved plot to: ./time_series\آتش‌نشانی_و_کلانتری.png
Found file: اسکان_موقت_1.txt

=== Processing file: اسکان_موقت_1.txt ===
Total lines read: 239
Found linear model: -1.472*t+3.870
Parsed slope: -1.472, intercept: 3.87
Parsed 231 data points.
Using trendline: y = -1.47*t + 3.87 | Annual Rate: -537 mm/year
✅ Saved plot to: ./time_series\اسکان_موقت_1.png
Found file: اسکان_موقت_2.txt

=== Processing f

In [ ]:
hi
look at this code :

import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import matplotlib.dates as mdates

# Set global font to Times New Roman
#plt.rcParams['font.family'] = 'Times New Roman'

# Directory containing time series files
directory = "./time_series"

# Function to parse date string to datetime
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y%m%d')
    except ValueError as e:
        print(f"Error parsing date '{date_str}': {e}")
        raise

# Function to process a single file and create a plot
def process_file(file_path, filename):
    print(f"Processing file: {file_path}")
    try:
        # Read the file
        dates = []
        displacements = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            print(f"Read {len(lines)} lines from {filename}")
            # Start reading data after the last '#' line
            data_start = False
            for line in lines:
                if line.startswith('#'):
                    data_start = True
                    continue
                if data_start and line.strip():
                    try:
                        date_str, displacement = line.strip().split()
                        dates.append(parse_date(date_str))
                        displacements.append(float(displacement))
                    except ValueError as e:
                        print(f"Error parsing line '{line.strip()}' in {filename}: {e}")
                        continue

        if not dates:
            print(f"No valid data found in {filename}")
            return

        print(f"Parsed {len(dates)} data points from {filename}")

        # Convert dates to matplotlib date numbers
        dates_mpl = mdates.date2num(dates)
        X = np.array(dates_mpl).reshape(-1, 1)
        y = np.array(displacements)

        # Fit linear model
        model = LinearRegression().fit(X, y)
        slope = model.coef_[0]
        intercept = model.intercept_
        trendline = model.predict(X)
        annual_rate = int(round(slope * 365))  # mm/year, without decimals
        print(f"Annual Rate: {annual_rate} mm/year")

        # Create the plot
        plt.figure(figsize=(14, 4))
        # Scatter plot with black outline
        plt.scatter(dates, y, color='red', label='InSAR Data', alpha=0.8, s=80, edgecolors='black', linewidth=0.3)
        plt.plot(dates, trendline, color='black', linestyle='--', alpha=0.6, linewidth=2.5, label=f'Annual Rate: {annual_rate} mm/year')

        # Set y-axis limits
        plt.ylim(-130, 30)

        # Labels and formatting
        plt.ylabel("Displacement (mm)", fontsize=14)
        plt.title(f"Subsidence", fontsize=15)
        plt.grid(True)
        # Place legend with trendline info
        plt.legend(fontsize=16, loc='best')

        # Rotate and bold x-axis tick labels
        plt.xticks(rotation=45)
        # for label in plt.gca().get_xticklabels():
        #     label.set_fontweight('bold')

        # Format x-axis to show only years
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())

        # Adjust layout to prevent label cutoff
        plt.tight_layout()

        # Save the plot
        output_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}.png")
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Saved plot to: {output_path}")
        plt.close()

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Check if directory exists
if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist")
else:
    print(f"Scanning directory: {directory}")
    # Iterate through all files in the time_series directory
    file_count = 0
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_count += 1
            file_path = os.path.join(directory, filename)
            print(f"Found file: {filename}")
            process_file(file_path, filename)
    if file_count == 0:
        print(f"No .txt files found in {directory}")

